In [53]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [54]:
path = Path('./corona_data_sets') # use your path
all_files = list(path.rglob("*.csv"))

In [62]:
d = {}

for p in all_files:
    df = pd.read_csv(p, index_col=None, header=0, converters={'ags2': str, 'ags5': str})
    df = df.drop(['bundesland', 'kreis', '_id'], 1)
    d[p.stem] = df

## firmenveraenderungen - company changes

In [63]:
company_changes = d["firmenveraenderungen"]
filter_col = [col for col in company_changes if col.startswith('d20')]
company_changes["value"] = company_changes[filter_col].mean(axis=1)
company_changes = company_changes.drop(filter_col, 1)
company_changes['value'] = company_changes['value'].astype(float)
#company_changes = company_changes.pivot_table(index=['ags2','ags5'], columns='variable', values='value')
company_changes


,ags2,ags5,variable,value
0,01,01001,kr_firm_br_a_m,5.821429
1,01,01002,kr_firm_br_a_m,24.464286
2,01,01003,kr_firm_br_a_m,65.535714
3,01,01004,kr_firm_br_a_m,18.428571
4,01,01051,kr_firm_br_a_m,359.250000
...,...,...,...,...
10023,16,16074,kr_firm_liq_m,1.750000
10024,16,16075,kr_firm_liq_m,1.285714
10025,16,16076,kr_firm_liq_m,1.964286
10026,16,16077,kr_firm_liq_m,1.214286


In [64]:
company_changes.pivot_table(index=['ags2','ags5'], columns='variable', values='value')


variable    kr_firm_br_99_m  kr_firm_br_a_m  kr_firm_br_b_m  kr_firm_br_c_m  \
ags2 ags5                                                                     
01   01001       126.392857        5.821429       -7.071429      168.178571   
     01002       334.964286       24.464286       -5.142857      464.500000   
     01003       288.535714       65.535714       -5.214286      400.857143   
     01004       111.750000       18.428571       -5.178571      166.357143   
     01051       257.821429      359.250000       -0.035714      298.892857   
...                     ...             ...             ...             ...   
16   16073       110.464286       96.250000       -1.250000      363.071429   
     16074        78.071429       79.107143       -3.321429      285.464286   
     16075       100.107143      101.500000       -1.678571      349.857143   
     16076       138.785714      133.928571       -4.285714      416.642857   
     16077        87.000000       83.678571        2.928571      228.392857   

variable    kr_firm_br_d_m  kr_firm_br_e_m  kr_firm_br_f_m  kr_firm_br_g_m  \
ags2 ags5                                                                    
01   01001       54.678571        2.392857      214.642857      807.250000   
     01002       39.285714       17.142857      584.571429     1773.500000   
     01003       16.464286       25.142857      609.107143     1626.250000   
     01004       -2.785714       12.642857      298.642857      705.071429   
     01051      278.785714       25.071429      578.321429     1062.678571   
...                    ...             ...             ...             ...   
16   16073        6.678571       16.035714      599.071429      855.821429   
     16074       12.285714        4.357143      504.928571      743.928571   
     16075        1.714286       14.071429      536.321429      769.142857   
     16076       18.714286       12.857143      721.178571      884.428571   
     16077        9.321429       16.392857      418.392857      660.678571   

variable    kr_firm_br_h_m  kr_firm_br_i_m  ...  kr_firm_br_o_m  \
ags2 ags5                                   ...                   
01   01001       98.285714      195.928571  ...       30.392857   
     01002      222.678571      483.357143  ...      118.535714   
     01003      325.321429      472.607143  ...       67.642857   
     01004       89.714286      144.785714  ...       30.642857   
     01051      175.250000      506.464286  ...      117.857143   
...                    ...             ...  ...             ...   
16   16073      106.928571      341.678571  ...      114.535714   
     16074      102.678571      184.035714  ...      109.928571   
     16075       96.428571      244.678571  ...      115.428571   
     16076      146.250000      244.000000  ...      107.321429   
     16077      117.428571      171.357143  ...       96.642857   

variable    kr_firm_br_p_m  kr_firm_br_q_m  kr_firm_br_r_m  kr_firm_br_s_m  \
ags2 ags5                                                                    
01   01001      106.142857      395.821429      116.500000      827.428571   
     01002      340.428571     1118.500000      296.642857     2360.035714   
     01003      216.785714      898.285714      249.392857     1991.464286   
     01004      110.678571      250.607143       95.250000      643.035714   
     01051      174.821429      477.928571      228.142857     1482.357143   
...                    ...             ...             ...             ...   
16   16073      136.142857      399.571429      117.785714     1122.285714   
     16074       99.107143      255.071429       89.178571      871.000000   
     16075      107.000000      288.107143      110.000000      965.678571   
     16076      131.250000      362.714286      131.785714     1099.392857   
     16077      109.500000      293.607143      108.428571      793.785714   

variable    kr_firm_br_t_m  kr_firm_br_u_m  kr_firm_liq_m  kr_firm_loe

In [65]:
#company_changes.to_csv('./corona_data_sets_modified/firmenveraenderungen.csv')


## ausbildungssituation - training situation

In [85]:
training_situation = d["ausbildungssituation"]
training_situation.rename(columns={"kr_schuel":"number_of_students"}, inplace=True)
training_situation = training_situation[["ags2","ags5","number_of_students"]]
training_situation

,ags2,ags5,number_of_students
0,01,01001,11610
1,01,01002,23898
2,01,01003,20478
3,01,01004,10380
4,01,01051,13158
...,...,...,...
396,16,16073,8772
397,16,16074,7190
398,16,16075,7441
399,16,16076,8403


In [79]:
#training_situation.to_csv('./corona_data_sets_modified/training_situation.csv')


## krankenhausdaten - hospital data


In [98]:
hospital_data = d["krankenhausdaten"]
hospital_data.rename(columns={"kr_kh":"number_of_hospitals",
                              "kr_kh_bett": "number_of_hospital_beds", 
                             "kr_kh_bett_ew":"number_of_hospital_beds_adj", 
                             "kr_khp": "hospital_patiants"}, 
                     inplace=True)
hospital_data

,ags2,ags5,number_of_hospitals,number_of_hospital_beds,number_of_hospital_beds_adj,kr_kh_bett_fab1,kr_kh_bett_fab2,kr_kh_bett_fab3,kr_kh_bett_fab4,kr_kh_bett_fab5,...,kr_khp_dg10_50u55,kr_khp_dg10_55u60,kr_khp_dg10_60u65,kr_khp_dg10_65u70,kr_khp_dg10_70u75,kr_khp_dg10_75u80,kr_khp_dg10_80u85,kr_khp_dg10_85u90,kr_khp_dg10_90u95,kr_khp_dg10_95
0,01,01001,3,860,9.6,-99,200,43,20,-99,...,68,104,88,142,161,171,100,51,35,5
1,01,01002,16,2498,10.1,57,496,116,80,57,...,171,152,235,237,288,296,274,165,84,18
2,01,01003,13,1975,9.1,42,431,92,54,61,...,132,199,243,311,348,485,335,275,158,41
3,01,01004,5,693,8.7,-99,182,32,7,-99,...,51,55,99,82,161,184,118,75,37,11
4,01,01051,3,656,4.9,-99,167,36,1,-99,...,100,111,121,192,201,245,181,120,53,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,16,16073,3,994,9.3,-99,243,53,7,82,...,99,115,140,125,168,230,214,147,65,8
397,16,16074,3,654,7.9,-99,20,-99,-99,-99,...,43,68,98,89,126,164,115,86,42,11
398,16,16075,1,120,1.5,-99,33,26,-99,-99,...,62,109,97,88,110,151,140,80,24,8
399,16,16076,2,403,4.1,-99,40,20,2,-99,...,70,79,147,108,144,189,186,121,71,12
